In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("Bronze-Analysis") \
    .config("spark.local.dir", "/spark_temp")\
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/10/11 03:49:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/11 03:49:17 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in standalone/kubernetes and LOCAL_DIRS in YARN).


In [2]:
df_accidents = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("./data/Accidents0515.csv")

df_casualties = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("./data/Casualties0515.csv")

df_vehicles = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("./data/Vehicles0515.csv")

print("Dados carregados")

Dados carregados


In [3]:
# CÉLULA 3 - Estatísticas Básicas
print("📊 RESUMO DOS DATASETS BRONZE:")
print(f"🚗 Acidentes: {df_accidents.count():,} linhas, {len(df_accidents.columns)} colunas")
print(f"👥 Vítimas: {df_casualties.count():,} linhas, {len(df_casualties.columns)} colunas")  
print(f"🚙 Veículos: {df_vehicles.count():,} linhas, {len(df_vehicles.columns)} colunas")

📊 RESUMO DOS DATASETS BRONZE:


🚗 Acidentes: 1,780,653 linhas, 32 colunas


👥 Vítimas: 2,402,909 linhas, 15 colunas


🚙 Veículos: 3,262,270 linhas, 22 colunas


In [4]:
print("🎯 AMOSTRA - Primeiros registros:")
print("\n--- ACIDENTES ---")
df_accidents.select("Accident_Index", "Accident_Severity", "Date", "Time").show(5)

print("\n--- VÍTIMAS ---")
df_casualties.select("Accident_Index", "Casualty_Severity", "Age_of_Casualty").show(5)

print("\n--- VEÍCULOS ---") 
df_vehicles.select("Accident_Index", "Vehicle_Type", "Age_of_Driver").show(5)

🎯 AMOSTRA - Primeiros registros:

--- ACIDENTES ---
+--------------+-----------------+----------+-------------------+
|Accident_Index|Accident_Severity|      Date|               Time|
+--------------+-----------------+----------+-------------------+
| 200501BS00001|                2|04/01/2005|2025-10-11 17:42:00|
| 200501BS00002|                3|05/01/2005|2025-10-11 17:36:00|
| 200501BS00003|                3|06/01/2005|2025-10-11 00:15:00|
| 200501BS00004|                3|07/01/2005|2025-10-11 10:35:00|
| 200501BS00005|                3|10/01/2005|2025-10-11 21:13:00|
+--------------+-----------------+----------+-------------------+
only showing top 5 rows

--- VÍTIMAS ---


+--------------+-----------------+---------------+
|Accident_Index|Casualty_Severity|Age_of_Casualty|
+--------------+-----------------+---------------+
| 200501BS00001|                2|             37|
| 200501BS00002|                3|             37|
| 200501BS00003|                3|             62|
| 200501BS00004|                3|             30|
| 200501BS00005|                3|             49|
+--------------+-----------------+---------------+
only showing top 5 rows

--- VEÍCULOS ---
+--------------+------------+-------------+
|Accident_Index|Vehicle_Type|Age_of_Driver|
+--------------+------------+-------------+
| 200501BS00001|           9|           74|
| 200501BS00002|          11|           42|
| 200501BS00003|          11|           35|
| 200501BS00003|           9|           62|
| 200501BS00004|           9|           49|
+--------------+------------+-------------+
only showing top 5 rows


In [5]:
df_accidents.write.mode("overwrite").option("compression", "snappy").parquet("./bronze/Accidents0515")
df_casualties.write.mode("overwrite").option("compression", "snappy").parquet("./bronze/Casualties0515")
df_vehicles.write.mode("overwrite").option("compression", "snappy").parquet("./bronze/Vehicles0515")

25/10/11 03:49:34 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


25/10/11 03:49:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/10/11 03:49:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/10/11 03:49:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/10/11 03:49:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/10/11 03:49:35 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


25/10/11 03:49:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/10/11 03:49:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/10/11 03:49:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/10/11 03:49:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/10/11 03:49:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/10/11 03:49:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/10/11 03:49:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/10/11 03:49:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/10/11 03:49:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


25/10/11 03:49:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers


25/10/11 03:49:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/10/11 03:49:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/10/11 03:49:46 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


25/10/11 03:49:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/10/11 03:49:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/10/11 03:49:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/10/11 03:49:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
25/10/11 03:49:49 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers


25/10/11 03:49:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers


25/10/11 03:49:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
25/10/11 03:49:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
25/10/11 03:49:52 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
